## Prepare imports

In [1]:
import matplotlib.pyplot as plt
from datasets import get_test_loader
from args import get_parser
from models.vae import BaseModel as VAE
from model_wrapper import TopologicalModelVAEPlainScaled
import torch

CATES = ["airplane"]

ModuleNotFoundError: No module named 'load_models'

In [19]:
p = get_parser()
args, unknown = p.parse_known_args()
args.cates = CATES

In [ ]:
# n_images = 4

# fig, axes = plt.subplots(
#     nrows=2, ncols=n_images, sharex=True, sharey=True, figsize=(8,4)
# )
# fig.subplots_adjust(wspace=0.05,hspace=0.05)

# for orig_img, recon_img,ax in zip(ect,decoded,axes.T):
    
#     ax[0].imshow(recon_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
#     ax[0].axis("off")
#     ax[1].imshow(orig_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
#     ax[1].axis("off")
    

# # plt.savefig("./figures/img/modelnet/reconstructed_ect_vae.svg",transparent=True)

In [ ]:
# n_images = 4

# fig, axes = plt.subplots(
#     nrows=2, ncols=n_images, sharex=True, sharey=True, figsize=(32,16)
# )
# fig.subplots_adjust(wspace=0.05,hspace=0.05)

# for orig_img, recon_img,ax in zip(ect,decoded,axes.T):
    
#     ax[0].imshow(recon_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
#     ax[0].axis("off")
#     ax[1].imshow(orig_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
#     ax[1].axis("off")

In [21]:
vae = VAE.load_from_checkpoint("./trained_models/vae_shapenet_{CATES[0]}")
model = TopologicalModelVAEPlainScaled(vae)
# Independent ECT layer
layer = vae.layer 



C:\Users\ernst\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightning\pytorch\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.3


  | Name                | Type             | Params
---------------------------------------------------------
0 | layer               | EctLayer         | 0     
1 | training_accuracy   | MeanSquaredError | 0     
2 | validation_accuracy | MeanSquaredError | 0     
3 | test_accuracy       | MeanSquaredError | 0     
4 | loss_fn             | MSELoss          | 0     
5 | model               | Sequential       | 39.9 M
---------------------------------------------------------
39.9 M    Trainable params
0         Non-trainable params
39.9 M    Total params
159.433   Total estimated model params size (MB)


In [22]:
print("=====GEN======")
loader = get_test_loader(args)
all_ect = []
all_recon_ect = []
for data in loader:
    idx_b, tr_pc, te_pc = data["idx"], data["train_points"], data["test_points"]

    te_pc = te_pc.cuda() if args.gpu is None else te_pc.cuda(args.gpu)
    tr_pc = tr_pc.cuda() if args.gpu is None else tr_pc.cuda(args.gpu)
    B, N = te_pc.size(0), te_pc.size(1)
    ect, recon_ect = model.reconstruct(tr_pc, num_points=N)

    all_ect.append(ect)
    all_recon_ect.append(recon_ect)

ref_ect = torch.cat(all_ect, dim=0)
recon_ect = torch.cat(all_recon_ect, dim=0)



=====GEN======
Total number of data:2832
Min number of points: (train)2048 (test)2048
Total number of data:405
Min number of points: (train)2048 (test)2048


# Reconstructions

In [ ]:
n_images = 4

fig, axes = plt.subplots(
    nrows=2, ncols=n_images, sharex=True, sharey=True, figsize=(8,4)
)
fig.subplots_adjust(wspace=0.05,hspace=0.05)

for orig_img, recon_img,ax in zip(ref_ect,recon_ect,axes.T):
    
    ax[0].imshow(recon_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
    ax[0].axis("off")
    ax[1].imshow(orig_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
    ax[1].axis("off")

# Samples 


In [ ]:
n_images = 4

samples = vae.sample(n_images, "cuda:0")


fig, axes = plt.subplots(
    nrows=1, ncols=n_images, sharex=True, sharey=True, figsize=(8,4)
)
fig.subplots_adjust(wspace=0.05,hspace=0.05)

for orig_img, recon_img,ax in zip(ref_ect,recon_ect,axes.T):
    
    ax[0].imshow(recon_img.cpu().detach().squeeze().numpy(),cmap="bone",vmin=-0.5,vmax=1.5)
    ax[0].axis("off")